In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import random

def weight(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1),name='W')
def bias(shape):
    return tf.Variable(tf.constant(0.1,shape=shape),name='b')
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

with tf.name_scope('input_layer'):
    x = tf.placeholder('float',shape=[None,32,32,3],name='x')
with tf.name_scope('conv_1'):
    W1 = weight([3,3,3,32])
    b1 = bias([32])
    conv_1 = conv2d(x,W1)+b1
    conv_1 = tf.nn.relu(conv_1)
    
with tf.name_scope('pool_1'):
    pool_1 = max_pool_2x2(conv_1)
    
with tf.name_scope('conv_2'):
    W2 = weight([3,3,32,64])
    b2 = bias([64])
    conv_2=conv2d(pool_1,W2)+b2
    conv_2=tf.nn.relu(conv_2)
    
with tf.name_scope('pool_2'):
    pool_2 = max_pool_2x2(conv_2)
    
with tf.name_scope('fc'):
    W3 = weight([4096,128])
    b3 = bias([128])
    flat = tf.reshape(pool_2,[-1,4096])
    h= tf.nn.relu(tf.matmul(flat,W3)+b3)
    h_dropout = tf.nn.dropout(h,keep_prob=0.8)
with tf.name_scope('output_layter'):
    W4 = weight([128,10])
    b4 = bias([10])
    pred = tf.nn.softmax(tf.matmul(h_dropout,W4)+b4,name='pred')

with tf.name_scope('optimizer'):
    y = tf.placeholder('float',shape=[None,10],name='label')
    loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=pred,labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss_function)
    

with tf.name_scope('evaluation'):
    correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,'float'))
    
sess = tf.Session()


saver = tf.train.Saver()

saver.restore(sess, "./cifar_50/cifar10_model.ckpt-50") 

%matplotlib inline

def load_CIFAR_batch(filename):
    with open(filename,'rb')as f:
        data_dict = pickle.load(f,encoding='bytes')
        images = data_dict[b'data']
        labels = data_dict[b'labels']
        
        #把元数据结构调整为：BCWH
        images = images.reshape(10000,3,32,32)
        #tesorflow 处理图像数据的结构为：BWHC
        #把通道数据c移动到最后一个维度
        images = images.transpose(0,2,3,1)
        labels = np.array(labels)
        return images,labels
test_image,test_label = load_CIFAR_batch('./cifar10_data/cifar-10-batches-py/test_batch')
print('显示原图',test_label[6])
plt.imshow(test_image[6])


import os
def load_CIFAR_data(data_dir):
    images_train = []
    labels_train = []


    for i in range(5):
        # 循环读取，每一个文件的数据和标签名
        f = os.path.join(data_dir,'data_batch_%d'%(i+1))
        print('loading',f)
        image_batch,label_batch=load_CIFAR_batch(f)
        images_train.append(image_batch)
        labels_train.append(label_batch)
        Xtrain=np.concatenate(images_train)
        Ytrain=np.concatenate(labels_train)
        del image_batch,label_batch
    Xtest,Ytest = load_CIFAR_batch(os.path.join(data_dir,'test_batch'))
    print("finished loadding CIFAR-10 data")
    return Xtrain,Ytrain,Xtest,Ytest
data_dir = './cifar10_data/cifar-10-batches-py/'

Xtrain,Ytrain,Xtest,Ytest = load_CIFAR_data(data_dir)


label_dict = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}
#定义显示图像数据以及对应标签的函数
def plot_images_labels_prediction(images,labels,prediction,idx,num=10):
    fig = plt.gcf()
    fig.set_size_inches(12,6)
    if num > 10:
        num=10
    for i in range(0,num):
        ax=plt.subplot(2,5,1+i)
        ax.imshow(images[idx],cmap='binary')
        
        title = str(i)+','+label_dict[labels[idx]]
        if len(prediction)>0:
            title+='=>'+label_dict[prediction[idx]]
        ax.set_title(title,fontsize=10)
        idx+=1
    plt.show()
    
plot_images_labels_prediction(Xtest,Ytest,[],1,10)

Xtest_normalize = Xtest.astype('float32')/255.0





/usr/local/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



ValueError: The passed save_path is not a valid checkpoint: ./cifar_50/cifar10_model.ckpt-50

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import random
%matplotlib inline

def load_CIFAR_batch(filename):
    with open(filename,'rb')as f:
        data_dict = pickle.load(f,encoding='bytes')
        images = data_dict[b'data']
        labels = data_dict[b'labels']
        
        #把元数据结构调整为：BCWH
        images = images.reshape(10000,3,32,32)
        #tesorflow 处理图像数据的结构为：BWHC
        #把通道数据c移动到最后一个维度
        images = images.transpose(0,2,3,1)
        labels = np.array(labels)
        return images,labels
test_image,test_label = load_CIFAR_batch('./cifar10_data/cifar-10-batches-py/test_batch')
print('显示原图',test_label[6])
plt.imshow(test_image[6])


import os
def load_CIFAR_data(data_dir):
    images_train = []
    labels_train = []


    for i in range(5):
        # 循环读取，每一个文件的数据和标签名
        f = os.path.join(data_dir,'data_batch_%d'%(i+1))
        print('loading',f)
        image_batch,label_batch=load_CIFAR_batch(f)
        images_train.append(image_batch)
        labels_train.append(label_batch)
        Xtrain=np.concatenate(images_train)
        Ytrain=np.concatenate(labels_train)
        del image_batch,label_batch
    Xtest,Ytest = load_CIFAR_batch(os.path.join(data_dir,'test_batch'))
    print("finished loadding CIFAR-10 data")
    return Xtrain,Ytrain,Xtest,Ytest
data_dir = './cifar10_data/cifar-10-batches-py/'

Xtrain,Ytrain,Xtest,Ytest = load_CIFAR_data(data_dir)


label_dict = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}
#定义显示图像数据以及对应标签的函数
def plot_images_labels_prediction(images,labels,prediction,idx,num=10):
    fig = plt.gcf()
    fig.set_size_inches(12,6)
    if num > 10:
        num=10
    for i in range(0,num):
        ax=plt.subplot(2,5,1+i)
        ax.imshow(images[idx],cmap='binary')
        
        title = str(i)+','+label_dict[labels[idx]]
        if len(prediction)>0:
            title+='=>'+label_dict[prediction[idx]]
        ax.set_title(title,fontsize=10)
        idx+=1
    plt.show()
    
plot_images_labels_prediction(Xtest,Ytest,[],1,10)

Xtest_normalize = Xtest.astype('float32')/255.0




sess = tf.Session()



saver =tf.train.import_meta_graph("cifar_50/cifar10_model.ckpt-50.meta")
saver.restore(sess, "./cifar_50/cifar10_model.ckpt-50") 

pred = tf.get_default_graph().get_tensor_by_name("pred:0")
x = tf.get_default_graph().get_tensor_by_name("x:0")

re = sess.run(pred,feed_dict={x:Xtest_normalize[:50]})



本文档写了两种取session的方式  一种需要完全重写一遍graph定义代码  且必须和源代码一致  
另一种则不需要 重写graph定义代码 
关于报取不到ckpt文件的错误  详情可见 https://www.cnblogs.com/monologuesmw/p/12273727.html 这篇博客写的很详细


In [ ]:
def predict(sess,inputs):
#     preds = sess.run('pred:0', feed_dict={'X:0': inputs}) 
#     test_pred = sess.run(pred,feed_dict={x:inputs[:50]})
    #部署clipper时用这种写法
    test_pred = sess.run('pred:0',feed_dict={'x:0':inputs[:50]})

    prediction_result =sess.run(tf.argmax(test_pred,1))
    # `X` is used, it must be defined in the model with that name explicitly!
    return prediction_result
# predict(sess,Xtest_normalize)
print(Xtest_normalize.shape)
print(Xtest_normalize[0].shape)
print(Xtest_normalize[0])


# predict(sess,Xtest_normalize)

In [ ]:

from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers.tensorflow import deploy_tensorflow_model
clipper_conn = ClipperConnection(DockerContainerManager())

In [ ]:
clipper_conn.start_clipper()

In [ ]:

clipper_conn.connect()

In [ ]:
clipper_conn.register_application(
    name="tf-app", input_type="doubles", default_output="-1.0", slo_micros=100000)

In [ ]:
deploy_tensorflow_model(
    clipper_conn,
    name="tf-mod",
    version=1,
    input_type="doubles",
    func=predict,
    tf_sess_or_saved_model_path=sess)

In [ ]:
clipper_conn.link_model_to_app(
    app_name="tf-app",
    model_name="tf-mod")

In [ ]:
query_address = clipper_conn.get_query_addr()

In [ ]:
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://"+query_address+"/tf-app/predict", headers=headers, data=json.dumps({
    "input": Xtest_normalize[0]})).json()

In [ ]:
clipper_conn.stop_all()